In [ ]:
import requests
import json
import time
import itertools
import os
from tqdm import tqdm
import sqlite3
import datetime
import pickle
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha
from Artha.twitter import TwitterAPI
from Artha.sqlitedb import TSQLite
import Artha.configs.twitter_config as c

checkra = TwitterAPI(username = "checkra_", bearer_token = c.c_bearer, key = c.c_key, secret = c.c_secret, token = c.c_token, token_secret = c.c_token_secret)
print(checkra.id,"done")

In [ ]:
checkra.user_lookup("BTC_JackSparrow")

In [ ]:
tweets = checkra.get_recent_tweets("BTC_JackSparrow")
print(len(tweets))

In [ ]:
# TODO Download recent tweets for all people I follow